In [151]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import io
import datetime
import statsmodels.formula.api as smf
import datetime

In [152]:
df = pd.read_csv('final_project_data.csv', parse_dates = True, index_col = 0)
df.head()

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
1989-03-31,9.668,NaN,NaN,NaN,NaN,NaN
1989-04-03,9.528,NaN,NaN,NaN,NaN,NaN
1989-04-04,9.405,NaN,NaN,NaN,NaN,NaN
1989-04-05,9.491,NaN,NaN,NaN,NaN,NaN
1989-04-06,9.526,NaN,NaN,NaN,NaN,NaN


In [153]:
#we find the first value of CPI in order the delet the NaN values that are before that date
initial_date = df['UK YoY CPI'].first_valid_index() 
#.fist_valid_index() returns the first non-NaN value
initial_date

Timestamp('2003-06-17 00:00:00')

In [154]:
#im gonna transform initial_date to first day of the last month 
initial_date = initial_date.replace(day=1)
#We reduce 1 month to this date
initial_date = initial_date - pd.DateOffset(months=1)
initial_date

Timestamp('2003-05-01 00:00:00')

In [155]:
#we are gonna drop all the values before initial_date
df = df[initial_date:]
df.head()

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-01,1.469,3.572,NaN,NaN,NaN,NaN
2003-05-02,1.565,3.631,NaN,NaN,NaN,NaN
2003-05-05,1.533,NaN,NaN,NaN,NaN,NaN
2003-05-06,1.428,3.644,NaN,NaN,NaN,NaN
2003-05-07,1.412,3.585,NaN,NaN,NaN,NaN


In [156]:
nan_values = df.isnull().sum()
print("NaN values in each column:")
print(nan_values)



NaN values in each column:
USD2YB                  299
UK2YB                   246
UK YoY CPI             5221
US YoY CPI             5220
US Industrial Index    5219
UK industrial Index    5220
dtype: int64


# Using .last

In [157]:
import pandas as pd

# Assuming df is your DataFrame with a datetime index and the specified columns

# Convert the index to datetime if needed
df.index = pd.to_datetime(df.index)

# Resample to monthly frequency and use .last() for CPI and Industrial Index
df = df.resample('M').last()

# For 'USD2YB' and 'UK2YB', multiply the values for each month
df['USD2YB'] = df.resample('M')['USD2YB'].prod() - 1
df['UK2YB'] = df.resample('M')['UK2YB'].prod() - 1

# Now, df_monthly contains the monthly data with .last() for CPI and Industrial Index
# and multiplication for 'USD2YB' and 'UK2YB'
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,0.326,2.450,NaN,2.2,90.4281,101.1
2003-06-30,0.304,2.485,1.2,2.1,90.4073,99.8
2003-07-31,0.744,2.818,1.1,2.1,90.5218,100.4
2003-08-31,0.972,3.167,1.3,2.1,90.9896,101.0
2003-09-30,0.462,3.015,1.4,2.2,90.7858,101.2
...,...,...,...,...,...,...
2023-09-30,4.046,3.887,6.7,3.7,103.2411,95.0
2023-10-31,4.089,3.766,6.7,3.7,103.3346,94.9
2023-11-30,3.682,3.589,4.6,3.2,102.4592,93.7


In [158]:
# Specify the columns to be shifted
columns_to_shift = ['UK YoY CPI', 'US YoY CPI', 'US Industrial Index', 'UK industrial Index']

# Shift the specified columns using .shift(-1)
df[columns_to_shift] = df[columns_to_shift].shift(-1)

# Display the resulting DataFrame with shifted columns
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,0.326,2.450,1.2,2.1,90.4073,99.8
2003-06-30,0.304,2.485,1.1,2.1,90.5218,100.4
2003-07-31,0.744,2.818,1.3,2.1,90.9896,101.0
2003-08-31,0.972,3.167,1.4,2.2,90.7858,101.2
2003-09-30,0.462,3.015,1.4,2.3,91.3829,101.4
...,...,...,...,...,...,...
2023-09-30,4.046,3.887,6.7,3.7,103.3346,94.9
2023-10-31,4.089,3.766,4.6,3.2,102.4592,93.7
2023-11-30,3.682,3.589,3.9,3.1,102.4301,93.9


In [159]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             2
US YoY CPI             2
US Industrial Index    1
UK industrial Index    2
dtype: int64

In [160]:
df = df.interpolate(method='linear')
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,0.326,2.450,1.2,2.1,90.4073,99.8
2003-06-30,0.304,2.485,1.1,2.1,90.5218,100.4
2003-07-31,0.744,2.818,1.3,2.1,90.9896,101.0
2003-08-31,0.972,3.167,1.4,2.2,90.7858,101.2
2003-09-30,0.462,3.015,1.4,2.3,91.3829,101.4
...,...,...,...,...,...,...
2023-09-30,4.046,3.887,6.7,3.7,103.3346,94.9
2023-10-31,4.089,3.766,4.6,3.2,102.4592,93.7
2023-11-30,3.682,3.589,3.9,3.1,102.4301,93.9


In [161]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

# using lambda x 

In [130]:
# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Resample the data to monthly frequency 
df = df.resample('M').apply(lambda x: (x + 1).prod() - 1)

# Display the resulting monthly data
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.400000,2.114286,91.455700,102.514286
2003-06-30,-0.792333,1.417429,1.336364,1.872727,91.841955,102.436364
2003-07-31,-0.548522,1.560870,1.300000,1.873913,92.156304,101.800000
2003-08-31,-0.172773,2.053364,1.361905,1.900000,92.260657,102.542857
2003-09-30,-0.306364,2.156227,1.334783,1.778261,92.643691,102.930435
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000


In [131]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

In [132]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

# using geometric mean

In [133]:
# Assuming df is your DataFrame with a datetime index
# and columns representing different financial indicators, including interest rates

# Convert the index to datetime if needed
df.index = pd.to_datetime(df.index)

# Define a custom aggregation function for interest rates using the geometric mean
def geometric_mean(arr):
    return np.exp(np.mean(np.log(1 + arr))) - 1

# Resample to monthly frequency and apply the aggregation function
df= df.resample('M').agg({
    'USD2YB': 'mean',               # Example non-interest rate column
    'UK2YB': 'mean',        # Example interest rate column
    'UK YoY CPI': 'mean',           # Add other non-interest rate columns
    'US YoY CPI': 'mean',
    'US Industrial Index': 'mean',
    'UK industrial Index': 'mean'
})

In [134]:
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.400000,2.114286,91.455700,102.514286
2003-06-30,-0.792333,1.417429,1.336364,1.872727,91.841955,102.436364
2003-07-31,-0.548522,1.560870,1.300000,1.873913,92.156304,101.800000
2003-08-31,-0.172773,2.053364,1.361905,1.900000,92.260657,102.542857
2003-09-30,-0.306364,2.156227,1.334783,1.778261,92.643691,102.930435
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000


In [135]:
# Specify the columns to be shifted
columns_to_shift = ['UK YoY CPI', 'US YoY CPI', 'US Industrial Index', 'UK industrial Index']

# Shift the specified columns using .shift(-1)
df[columns_to_shift] = df[columns_to_shift].shift(-1)

# Display the resulting DataFrame with shifted columns
df


,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.336364,1.872727,91.841955,102.436364
2003-06-30,-0.792333,1.417429,1.300000,1.873913,92.156304,101.800000
2003-07-31,-0.548522,1.560870,1.361905,1.900000,92.260657,102.542857
2003-08-31,-0.172773,2.053364,1.334783,1.778261,92.643691,102.930435
2003-09-30,-0.306364,2.156227,1.190909,1.700000,92.661518,102.863636
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000


In [136]:
df.isnull().sum()


USD2YB                 0
UK2YB                  0
UK YoY CPI             1
US YoY CPI             1
US Industrial Index    1
UK industrial Index    1
dtype: int64

In [137]:
df = df.interpolate(method='linear')
df

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,-0.616739,1.538217,1.336364,1.872727,91.841955,102.436364
2003-06-30,-0.792333,1.417429,1.300000,1.873913,92.156304,101.800000
2003-07-31,-0.548522,1.560870,1.361905,1.900000,92.260657,102.542857
2003-08-31,-0.172773,2.053364,1.334783,1.778261,92.643691,102.930435
2003-09-30,-0.306364,2.156227,1.190909,1.700000,92.661518,102.863636
...,...,...,...,...,...,...
2023-09-30,3.038045,2.983545,4.250000,3.120000,102.447130,93.820000
2023-10-31,3.083091,2.851818,4.250000,3.120000,102.447130,93.820000
2023-11-30,2.893478,2.614522,4.250000,3.120000,102.447130,93.820000


In [138]:
df.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64